In [1]:
# importing pandas
import pandas as pd

In [2]:
# importing and viewing the data
df = pd.read_csv('./data/apple_stock_data.csv')
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [3]:
# converting the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
# checking the conversion effectiveness
df['Date'].dtype

datetime64[ns, UTC]

In [5]:
# setting the 'Date' column as the dataframe index
df.set_index('Date', inplace=True)

In [6]:
#checking the dataframe indexes and their type
df.index

DatetimeIndex(['2023-11-02 00:00:00+00:00', '2023-11-03 00:00:00+00:00',
               '2023-11-06 00:00:00+00:00', '2023-11-07 00:00:00+00:00',
               '2023-11-08 00:00:00+00:00', '2023-11-09 00:00:00+00:00',
               '2023-11-10 00:00:00+00:00', '2023-11-13 00:00:00+00:00',
               '2023-11-14 00:00:00+00:00', '2023-11-15 00:00:00+00:00',
               ...
               '2024-10-21 00:00:00+00:00', '2024-10-22 00:00:00+00:00',
               '2024-10-23 00:00:00+00:00', '2024-10-24 00:00:00+00:00',
               '2024-10-25 00:00:00+00:00', '2024-10-28 00:00:00+00:00',
               '2024-10-29 00:00:00+00:00', '2024-10-30 00:00:00+00:00',
               '2024-10-31 00:00:00+00:00', '2024-11-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=252, freq=None)

In [7]:
# reducing the dataframe to the 'Close' column
df = df[['Close']]

In [8]:
# checking the new form of the dataframe
df.head()

,Close
Date,
2023-11-02 00:00:00+00:00,177.570007
2023-11-03 00:00:00+00:00,176.649994
2023-11-06 00:00:00+00:00,179.229996
2023-11-07 00:00:00+00:00,181.820007
2023-11-08 00:00:00+00:00,182.889999


## Choosing the Hybrid Models


    We will be using LSTM (Long Short-Term Memory) and Linear Regression models for this task. I chose LSTM because it effectively captures sequential dependencies and patterns in time-series data, which makes it suitable for modelling stock price movements influenced by historical trends.

    Linear Regression, on the other hand, is a straightforward model that captures simple linear relationships and long-term trends in data. By combining these two models into a hybrid approach, we leverage the LSTM’s ability to model complex time-dependent patterns alongside the Linear Regression’s ability to identify and follow broader trends. This combination aims to create a more balanced and accurate prediction system.

### Data normalization


In [9]:
# importing required library
from sklearn.preprocessing import MinMaxScaler

In [10]:
# declaring and initializing the scaler
scaler = MinMaxScaler(feature_range=(0,1))

In [11]:
# applying the scaler to the data
df['Close'] = scaler.fit_transform(df[['Close']])

In [12]:
# checking the result
df['Close'].head()

Date
2023-11-02 00:00:00+00:00    0.175853
2023-11-03 00:00:00+00:00    0.162983
2023-11-06 00:00:00+00:00    0.199077
2023-11-07 00:00:00+00:00    0.235311
2023-11-08 00:00:00+00:00    0.250280
Name: Close, dtype: float64

### Preparing data for LSTM

#### Creating sequences of a defined length(e.g: 60days)


In [13]:
# importing the required library
import numpy as np

In [14]:
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    
    return np.array(X), np.array(y)

In [15]:
seq_length = 60
X, y = create_sequences(df['Close'].values, seq_length)

In [16]:
X.shape

(192, 60)

In [17]:
y.shape

(192,)

In [18]:
#### Splitting the sequences into training and test sets(80-20)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [19]:
X_train.shape

(153, 60)

In [20]:
y_train.shape

(153,)

#### Building a sequential LSTM model with layers to capture the temporal dependencies in the data

In [21]:
# importing required library
import tensorflow as tf
import keras
from keras import layers

2025-01-17 15:32:31.435398: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 15:32:31.436021: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 15:32:31.438185: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-17 15:32:31.443766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737124351.453861  282085 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737124351.45

In [22]:
lstm_model = keras.Sequential()
lstm_model.add(layers.Input(shape = (X_train.shape[1], 1)))
lstm_model.add(layers.LSTM(units=50, return_sequences=True))
lstm_model.add(layers.LSTM(units=50))
lstm_model.add(layers.Dense(1))

2025-01-17 15:32:33.025985: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [23]:
# compiling the model 
lstm_model.compile(optimizer='adam',loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.1683
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0317
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0254
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0213
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0194
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0135
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0145
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0111
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0121
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0098
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0102
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0095
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0096
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0091
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0094
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━

In [24]:
df

,Close
Date,
2023-11-02 00:00:00+00:00,0.175853
2023-11-03 00:00:00+00:00,0.162983
2023-11-06 00:00:00+00:00,0.199077
2023-11-07 00:00:00+00:00,0.235311
2023-11-08 00:00:00+00:00,0.250280
...,...
2024-10-28 00:00:00+00:00,0.956911
2024-10-29 00:00:00+00:00,0.960688
2024-10-30 00:00:00+00:00,0.910744


In [25]:
# training the second model(Linear Regression)
# generating lagged features for Linear Regression (e.g., using the past 3 days as predictors)
df['Lag_1'] = df['Close'].shift(1)
df['Lag_2'] = df['Close'].shift(2)
df['Lag_3'] = df['Close'].shift(3)
#df = df.dropna()
#df  = df.fillna(method='ffill')

In [26]:
df

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2023-11-02 00:00:00+00:00,0.175853,NaN,NaN,NaN
2023-11-03 00:00:00+00:00,0.162983,0.175853,NaN,NaN
2023-11-06 00:00:00+00:00,0.199077,0.162983,0.175853,NaN
2023-11-07 00:00:00+00:00,0.235311,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.250280,0.235311,0.199077,0.162983
...,...,...,...,...
2024-10-28 00:00:00+00:00,0.956911,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.960688,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.910744,0.960688,0.956911,0.929071


In [27]:
df = df.dropna()

In [28]:
df

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2023-11-07 00:00:00+00:00,0.235311,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.250280,0.235311,0.199077,0.162983
2023-11-09 00:00:00+00:00,0.243565,0.250280,0.235311,0.199077
2023-11-10 00:00:00+00:00,0.299384,0.243565,0.250280,0.235311
2023-11-13 00:00:00+00:00,0.277001,0.299384,0.243565,0.250280
...,...,...,...,...
2024-10-28 00:00:00+00:00,0.956911,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.960688,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.910744,0.960688,0.956911,0.929071


In [29]:
# checking the result
df.head()

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2023-11-07 00:00:00+00:00,0.235311,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.250280,0.235311,0.199077,0.162983
2023-11-09 00:00:00+00:00,0.243565,0.250280,0.235311,0.199077
2023-11-10 00:00:00+00:00,0.299384,0.243565,0.250280,0.235311
2023-11-13 00:00:00+00:00,0.277001,0.299384,0.243565,0.250280


In [30]:
# splitting the data accordingly for training and testing
X_lin = df[['Lag_1', 'Lag_2', 'Lag_3']]
y_lin = df['Close']
X_train_lin, X_test_lin = X_lin[:train_size], X_lin[train_size:]
y_train_lin, y_test_lin = y_lin[:train_size], y_lin[train_size:]

In [31]:
X_lin

,Lag_1,Lag_2,Lag_3
Date,,,
2023-11-07 00:00:00+00:00,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.235311,0.199077,0.162983
2023-11-09 00:00:00+00:00,0.250280,0.235311,0.199077
2023-11-10 00:00:00+00:00,0.243565,0.250280,0.235311
2023-11-13 00:00:00+00:00,0.299384,0.243565,0.250280
...,...,...,...
2024-10-28 00:00:00+00:00,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.960688,0.956911,0.929071


In [32]:
y_lin

Date
2023-11-07 00:00:00+00:00    0.235311
2023-11-08 00:00:00+00:00    0.250280
2023-11-09 00:00:00+00:00    0.243565
2023-11-10 00:00:00+00:00    0.299384
2023-11-13 00:00:00+00:00    0.277001
                               ...   
2024-10-28 00:00:00+00:00    0.956911
2024-10-29 00:00:00+00:00    0.960688
2024-10-30 00:00:00+00:00    0.910744
2024-10-31 00:00:00+00:00    0.852127
2024-11-01 00:00:00+00:00    0.810157
Name: Close, Length: 249, dtype: float64

In [33]:
# training the linear regression model
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train_lin, y_train_lin)

LinearRegression()

### Making predictions using LSTM on the test set and inverse transform the scaled predictions

In [34]:
# reshaping the test set
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# making predictions
lstm_predictions = lstm_model.predict(X_test_lstm)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


In [35]:
# checking the predictions
lstm_predictions

array([[0.83487797],
       [0.8232641 ],
       [0.8134177 ],
       [0.80535036],
       [0.79874873],
       [0.7887109 ],
       [0.7772869 ],
       [0.76851887],
       [0.7683855 ],
       [0.77369887],
       [0.781268  ],
       [0.7907661 ],
       [0.80031085],
       [0.8102829 ],
       [0.8201947 ],
       [0.8335698 ],
       [0.84324574],
       [0.85036844],
       [0.85440594],
       [0.85693854],
       [0.85424095],
       [0.8512985 ],
       [0.85115844],
       [0.8525076 ],
       [0.8536741 ],
       [0.85758346],
       [0.8650816 ],
       [0.8730087 ],
       [0.8811162 ],
       [0.89107656],
       [0.90283614],
       [0.9146616 ],
       [0.92190826],
       [0.9254853 ],
       [0.9269052 ],
       [0.9282125 ],
       [0.9294918 ],
       [0.9278801 ],
       [0.9211453 ]], dtype=float32)

In [36]:

# inverse transforming the scaled predictions
lstm_predictions = scaler.inverse_transform(lstm_predictions)

In [37]:
# checking the predictions after inverse transforming
lstm_predictions

array([[224.67708],
       [223.84692],
       [223.1431 ],
       [222.56644],
       [222.09456],
       [221.37704],
       [220.56046],
       [219.93373],
       [219.9242 ],
       [220.30399],
       [220.84503],
       [221.52396],
       [222.20622],
       [222.91902],
       [223.62752],
       [224.58357],
       [225.2752 ],
       [225.78433],
       [226.07294],
       [226.25397],
       [226.06114],
       [225.85081],
       [225.8408 ],
       [225.93724],
       [226.02063],
       [226.30006],
       [226.83603],
       [227.40266],
       [227.98218],
       [228.69415],
       [229.53473],
       [230.38002],
       [230.898  ],
       [231.15369],
       [231.25517],
       [231.34862],
       [231.44006],
       [231.32486],
       [230.84346]], dtype=float32)

In [38]:
# using the Linear regression model to make predictions
lin_predictions = lin_model.predict(X_test_lin)

In [39]:
# checking the predictions
lin_predictions

array([0.72493302, 0.69069753, 0.62133437, 0.59129816, 0.60368019,
       0.61842157, 0.67892883, 0.6902931 , 0.63702579, 0.72640727,
       0.7790112 , 0.79376088, 0.86148775, 0.8817707 , 0.89140456,
       0.95319754, 0.87324617, 0.91478365, 0.97383401, 0.97779832,
       0.88887159, 0.82077387, 0.82663263, 0.82419342, 0.8394823 ,
       0.74514936, 0.73001466, 0.74069391, 0.74532185, 0.75319058,
       0.80082198, 0.7458247 , 0.76629047, 0.61371113, 0.584374  ,
       0.62845921, 0.68023277, 0.72108285, 0.73755725, 0.79029913,
       0.79567466, 0.83725846, 0.85604375, 0.85183487, 0.86014145,
       0.8585525 , 0.83092634, 0.86492186, 0.87067005, 0.88171257,
       0.85894488, 0.90679742, 0.89574908, 0.8030576 , 0.77624974,
       0.80236425, 0.78051241, 0.78114142, 0.77034811, 0.80760153,
       0.80935377, 0.80394486, 0.71374111, 0.72150312, 0.78188034,
       0.90049478, 0.8874991 , 0.85766748, 0.87139203, 0.85767925,
       0.87406686, 0.87843766, 0.95376874, 0.85390778, 0.86007

In [40]:
# inverse transforming(denormalization) the predictions
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1,1))

In [41]:
# checking the predictions
lin_predictions

array([[216.81820901],
       [214.37105639],
       [209.41297789],
       [207.26599011],
       [208.15105726],
       [209.20477132],
       [213.52982995],
       [214.34214776],
       [210.534601  ],
       [216.92358859],
       [220.68371689],
       [221.73802455],
       [226.5791409 ],
       [228.02896608],
       [228.71759398],
       [233.13455589],
       [227.41963259],
       [230.38873171],
       [234.60965097],
       [234.89301977],
       [228.53653766],
       [223.66891254],
       [224.08769717],
       [223.91334202],
       [225.00619103],
       [218.26327302],
       [217.18144479],
       [217.94479768],
       [218.27560289],
       [218.8380597 ],
       [222.24275147],
       [218.31154614],
       [219.77443979],
       [208.86806873],
       [206.77105101],
       [209.92226181],
       [213.62303576],
       [216.54299919],
       [217.72058886],
       [221.49057849],
       [221.87482097],
       [224.84723133],
       [226.19000353],
       [225

In [42]:
lstm_predictions.shape

(39, 1)

In [43]:
lin_predictions.shape

(96, 1)

In [44]:
# making hybrid predictions using the previous predictions
#hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)

### Predicting using the hybrid model

In [45]:
# predicting using the LSTM model

lstm_future_predictions = []
last_sequence = X_test[-1].reshape(1, seq_length, 1)

for _ in range(10):
    lstm_pred = lstm_model.predict(last_sequence)[0,0]
    lstm_future_predictions.append(lstm_pred)
    lstm_pred_reshaped = np.array([[lstm_pred]]).reshape(1,1,1)
    last_sequence = np.append(last_sequence[:,1:,:], lstm_pred_reshaped, axis=1)

lstm_future_predictions = scaler.inverse_transform(np.array(lstm_future_predictions).reshape(-1,1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


In [46]:
X_test_lin

,Lag_1,Lag_2,Lag_3
Date,,,
2024-06-18 00:00:00+00:00,0.722860,0.664382,0.688864
2024-06-20 00:00:00+00:00,0.689563,0.722860,0.664382
2024-06-21 00:00:00+00:00,0.625070,0.689563,0.722860
2024-06-24 00:00:00+00:00,0.594432,0.625070,0.689563
2024-06-25 00:00:00+00:00,0.603525,0.594432,0.625070
...,...,...,...
2024-10-28 00:00:00+00:00,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.960688,0.956911,0.929071


In [47]:
# predicting using the Linear Regression
recent_data = df['Close'].values[-3:]
lin_future_predictions = []

for _ in range(10):
    lin_pred = lin_model.predict(recent_data.reshape(1,-1))[0]
    lin_future_predictions.append(lin_pred)
    recent_data = np.append(recent_data[1:], lin_pred)

lin_future_predictions = scaler.inverse_transform(np.array(lin_future_predictions).reshape(-1,1))

/home/cedric-hazoume/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/cedric-hazoume/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/cedric-hazoume/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/cedric-hazoume/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home/cedric-hazoume/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/home

In [48]:
# combining both models
hybrid_future_predictions = (0.7 * lstm_future_predictions) + (0.3 * lin_future_predictions)

In [49]:
# creating a dataframe to look at the predictions
future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=10)

predictions_df = pd.DataFrame({
    'Date': future_dates,
    'LSTM predictions': lstm_future_predictions.flatten(),
    'Linear Regression Predictions': lin_future_predictions.flatten(),
    'Hybrid Model Predictions': hybrid_future_predictions.flatten()
})

In [50]:
predictions_df

,Date,LSTM predictions,Linear Regression Predictions,Hybrid Model Predictions
0,2024-11-02 00:00:00+00:00,230.843460,230.355192,230.696981
1,2024-11-03 00:00:00+00:00,230.410538,225.707291,228.999556
2,2024-11-04 00:00:00+00:00,229.999359,222.703426,227.810570
3,2024-11-05 00:00:00+00:00,229.602631,230.631535,229.911293
4,2024-11-06 00:00:00+00:00,229.215576,225.486380,228.096811
5,2024-11-07 00:00:00+00:00,228.834778,222.494588,226.932718
6,2024-11-08 00:00:00+00:00,228.458069,230.930195,229.199698
7,2024-11-09 00:00:00+00:00,228.084091,225.245599,227.232539
8,2024-11-10 00:00:00+00:00,227.712158,222.284007,226.083716
9,2024-11-11 00:00:00+00:00,227.341995,231.252375,228.515102


In [51]:
df.tail()

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2024-10-28 00:00:00+00:00,0.956911,0.929071,0.917320,0.919978
2024-10-29 00:00:00+00:00,0.960688,0.956911,0.929071,0.917320
2024-10-30 00:00:00+00:00,0.910744,0.960688,0.956911,0.929071
2024-10-31 00:00:00+00:00,0.852127,0.910744,0.960688,0.956911
2024-11-01 00:00:00+00:00,0.810157,0.852127,0.910744,0.960688


In [52]:
df.head()

,Close,Lag_1,Lag_2,Lag_3
Date,,,,
2023-11-07 00:00:00+00:00,0.235311,0.199077,0.162983,0.175853
2023-11-08 00:00:00+00:00,0.250280,0.235311,0.199077,0.162983
2023-11-09 00:00:00+00:00,0.243565,0.250280,0.235311,0.199077
2023-11-10 00:00:00+00:00,0.299384,0.243565,0.250280,0.235311
2023-11-13 00:00:00+00:00,0.277001,0.299384,0.243565,0.250280
